In [1]:
import numpy as np
import pandas as pd

1. load the file
2. create universe dataframe
3. create buy sell signal
4. price all options

In [89]:
df_spx=pd.read_csv('SPX.csv')  
df_spx['date']=pd.to_datetime(df_spx['date'])
df_spx.index=df_spx['date']
df_spx=df_spx.sort_index()

In [90]:
df_bvols=pd.read_excel('bloomberg_spx_vols_values.xlsx', index_col=0) 
#df_bvols['Date']=pd.to_datetime(df_bvols['Date'])
#df_bvols.index=df_bvols['date']
#df_bvols=df_bvols.sort_index()
df_bvols=df_bvols['2005-05-18':]
df_bvols

,px,12m atm vol,24m atm vol,12m slope,24m slope
date,,,,,
2005-05-18,1185.56,14.7994,16.1761,-0.830764,-2.561531
2005-05-19,1191.08,14.4530,15.7034,-0.832010,-2.310149
2005-05-20,1189.28,14.3018,15.6604,-0.847508,-2.447997
2005-05-23,1193.86,14.0931,15.3672,-0.861660,-2.837509
2005-05-24,1194.07,14.0674,15.4045,-0.840980,-2.289923
...,...,...,...,...,...
2020-10-15,3483.34,22.2533,21.6859,-3.730241,-3.464660
2020-10-16,3483.81,22.5848,21.8534,-3.816402,-3.597786
2020-10-19,3426.92,23.4197,22.4914,-3.765921,-3.487846


In [125]:
pct_strikes=np.array([0.92,1,1,1.08])
maturity_days=720
maturity_years=2
price_field="px"
vol12_field="12m atm vol"
vol24_field="24m atm vol"
slope12_field="12m slope"
slope24_field="24m slope"
df_bvols[vol12_field]=df_bvols[vol12_field]/100
df_bvols[vol24_field]=df_bvols[vol24_field]/100
df_bvols[slope12_field]=df_bvols[slope12_field]/100
df_bvols[slope24_field]=df_bvols[slope24_field]/100

df_bvols.replace(0, np.nan, inplace=True)
df_bvols=df_bvols.fillna(method='ffill')
df_option_universe=pd.DataFrame(index=df_bvols.index)
strikes_universe=[]
creation_dates_universe=[]
maturity_dates_universe=[]
pc_universe=[]

for ind, row in df_bvols.iterrows():
    strikes=pct_strikes*row[price_field]
    pc=['p']*2+['c']*2
    creation_dates=[ind]*4
    maturity_dates=[ind+pd.Timedelta(days=maturity_years*365)]*4
    maturity_dates=pd.to_datetime(maturity_dates)
    strikes_universe.extend(strikes)
    creation_dates_universe.extend(creation_dates)
    maturity_dates_universe.extend(maturity_dates)
    pc_universe.extend(pc)

    #df_option_universe.loc[ind,'strikes']=strikes
    
    #if ind.day==1:
    #    df_spx.loc[ind,'pct_strike']=pct_strikes[np.random.randint(0,4)]
    #    df_spx.loc[ind,'maturity_days']=maturity_days
    #    maturity_days+=90


# create grids of strikes, creation dates, maturity dates, and pc]

strikes_universe=np.array(strikes_universe)
creation_dates_universe=np.array(creation_dates_universe)
maturity_dates_universe=np.array(maturity_dates_universe)
pc_universe=np.array(pc_universe)

#create 2d grids of strikes, creation dates, maturity dates, and pc where one dimension is the index of bvols and the other is the index of the strike
strikes_grid=np.tile(strikes_universe, (len(df_bvols),1))
creation_dates_grid=np.tile(creation_dates_universe, (len(df_bvols),1))
#need to figure out whiy the following needed to be done (convert back to datetime)
maturity_dates_grid=np.tile(maturity_dates_universe, (len(df_bvols),1))
pc_grid=np.tile(pc_universe, (len(df_bvols),1))
vol12_grid=np.tile(df_bvols[vol12_field].values, (len(strikes_universe),1)).T
vol24_grid=np.tile(df_bvols[vol24_field].values, (len(strikes_universe),1)).T
slope12_grid=np.tile(df_bvols[slope12_field].values, (len(strikes_universe),1)).T
slope24_grid=np.tile(df_bvols[slope24_field].values, (len(strikes_universe),1)).T
date_grid=np.tile(df_bvols.index.to_numpy(), (len(strikes_universe),1)).T

In [127]:
myfunc_vec = np.vectorize(lambda x: x.to_datetime64())
texp_grid=(myfunc_vec(maturity_dates_grid) -date_grid)/np.timedelta64(1, 'D')/365